#### If using AMD GPU, switch backend to PlaidML library:

In [14]:
import os
os.environ['KERAS_BACKEND']='plaidml.keras.backend'

In [15]:
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.utils import Sequence
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, ZeroPadding2D, BatchNormalization
from tensorflow.keras.callbacks import TensorBoard
from keras_preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
import time
import pickle
from keras.models import load_model

#### Define Experiments

In [25]:
conv_layers = [5]      # number of conv layers
layer_sizes = [16]     # number of nodes in a layer
dense_layers = [2]     # number of dense layers

#### Load input data

In [17]:
pickle_in = open('../Dataset/df_all.pickle', 'rb')
df_train, df_test = pickle.load(pickle_in)

In [18]:
# The Keras ImageDataGenerator uses string type data label
df_train['gender'] = df_train.gender.astype(str)
df_test['gender'] = df_test.gender.astype(str)

In [19]:
print(df_train.shape, df_test.shape)

(196464, 10) (10340, 10)


In [20]:
df_train.dtypes

path                object
id                  uint16
name                object
dob         datetime64[ns]
gender              object
score1             float64
score2             float64
pic_date    datetime64[ns]
region              object
age                float64
dtype: object

We will be using a generator to feed model with images, the X would be the path to these images. y will be the gender label.

In [21]:
image_reshape_size = 100
input_image_root_dir = '../Dataset/imdb_crop/' # Don't forget the ending slash

In [22]:
from keras import backend as K
K.set_image_data_format('channels_last')
batch_size = 64
inputShape = (image_reshape_size, image_reshape_size, 1)

In [23]:
df_train.head(3)

,path,id,name,dob,gender,score1,score2,pic_date,region,age
162333,66/nm0005166_rm623219712_1946-12-23_2005.jpg,18382,Susan Lucci,1946-12-23,0,0.969644,NaN,2005-01-01,"[0.914, 95.97000000000001, 65.808, 160.864]",58.027201
130252,23/nm0001823_rm3503795712_1949-12-7_2012.jpg,19153,Tom Waits,1949-12-07,1,3.493649,NaN,2012-01-01,"[529.0156677515998, 416.0940246619713, 865.732...",62.068352
390320,78/nm0273178_rm3028849152_1921-8-28_2006.jpg,6300,Fernando Fernán Gómez,1921-08-28,1,0.782722,NaN,2006-01-01,"[46.70229827255775, 37.4382386180462, 129.6968...",84.346701


#### Set up input image generator using flow_from_dataframe

In [24]:
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.1)

train_generator = datagen.flow_from_dataframe(dataframe=df_train,
                                            directory=input_image_root_dir,
                                            x_col="path", y_col="gender",
                                            subset="training",
                                            class_mode="binary",
                                            color_mode="grayscale",
                                            target_size=(image_reshape_size,image_reshape_size),
                                            batch_size=batch_size,
                                            seed=1,
                                            shuffle=True)

val_generator = datagen.flow_from_dataframe(dataframe=df_train,
                                            directory=input_image_root_dir,
                                            x_col="path", y_col="gender",
                                            subset="validation",
                                            class_mode="binary",
                                            color_mode="grayscale",
                                            target_size=(image_reshape_size,image_reshape_size),
                                            batch_size=batch_size,
                                            seed=1,
                                            shuffle=True)

test_generator = datagen.flow_from_dataframe(dataframe=df_test, 
                                            directory=input_image_root_dir, 
                                            x_col="path", y_col=None, 
                                            class_mode=None, 
                                            color_mode="grayscale",
                                            target_size=(image_reshape_size,image_reshape_size),
                                            batch_size=1,
                                            shuffle=False)

Found 176818 images belonging to 2 classes.
Found 19646 images belonging to 2 classes.
Found 10340 images.


#### (1) Run training experiments

In [26]:
for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            
            NAME = 'BN-{}-conv-{}-node-{}-dens-{}'.format(conv_layer, layer_size, dense_layer, int(time.time()))  # model name with timestamp
            print(NAME) 
            
            tensorboard = TensorBoard(log_dir='logs/{}'.format(NAME))
            checkpoint = ModelCheckpoint('weights/{}'.format(NAME), monitor='val_loss', verbose=0, save_best_only=False, save_weights_only=False, mode='auto', period=1)
            callbacks = [tensorboard, checkpoint]
            
            model = Sequential()
            
            # first layer
            model.add(Conv2D(layer_size, (3,3), padding="same", activation="relu", input_shape=inputShape))
            model.add(BatchNormalization())
            model.add(MaxPooling2D(pool_size=(3,3)))
            
            # sets up additional # of conv layers
            for _ in range(conv_layer - 1):
                layer_size *= 2
                model.add(Conv2D(layer_size, (3,3), padding="same", activation="relu"))
                model.add(BatchNormalization())
                model.add(Conv2D(layer_size, (3,3), padding="same", activation="relu"))
                model.add(BatchNormalization())
                model.add(MaxPooling2D(pool_size=(2,2)))
                model.add(Dropout(0.25))
            
            model.add(Flatten())
            
            layer_size *= 4 # to get the dense layer to be 8X of last output size
            
            # sets up # of dense layers
            for _ in range(dense_layer):
                model.add(Dense(layer_size, activation='relu'))
                model.add(BatchNormalization())
                model.add(Dropout(0.5))
            
            # output layer
            model.add(Dense(1))
            model.add(Activation('sigmoid'))
            
            opt = Adam(lr=0.001)
            model.compile(loss='binary_crossentropy', 
                          optimizer=opt,
                          metrics=['accuracy'])
            
            # Optional for resuming, load weights
#             model.load_weights('weights/BN-5-conv-32-node-2-dens-1554396563')

            model.fit_generator(generator=train_generator,
                                steps_per_epoch=(train_generator.n // train_generator.batch_size),
                                callbacks = callbacks,
                                validation_data=val_generator,
                                validation_steps=(val_generator.n // val_generator.batch_size),
                                epochs=20,
                                use_multiprocessing=False,
                                workers=4)
            
            filepath = NAME + '.h5'
            model.save(filepath)

BN-5-conv-16-node-2-dens-1554487860


INFO:plaidml:Opening device "opencl_intel_intel(r)_hd_graphics_630.0"


Epoch 1/20


INFO:plaidml:Analyzing Ops: 169 of 1652 operations complete
INFO:plaidml:Analyzing Ops: 594 of 1652 operations complete
INFO:plaidml:Analyzing Ops: 670 of 1652 operations complete
INFO:plaidml:Analyzing Ops: 705 of 1652 operations complete


 457/2762 [===>..........................] - ETA: 55:58 - loss: 0.7592 - acc: 0.6058

KeyboardInterrupt: 

#### Load model, resume training

In [ ]:
inputFile = 'BN-5-conv-32-node-2-dens-1553895953-3.h5'
model = load_model(inputFile)

In [ ]:
saveAs = 'BN-5-conv-32-node-2-dens-1553895953-4Lab'

In [ ]:
from keras.callbacks import ModelCheckpoint

NAME = saveAs
print(NAME) 

tensorboard = TensorBoard(log_dir='logs/{}'.format(NAME))
callbacks = [tensorboard]

opt = Adam(lr=0.001)
model.compile(loss='binary_crossentropy', 
              optimizer=opt,
              metrics=['accuracy'])

model.fit_generator(generator=train_generator,
                    steps_per_epoch=(train_generator.n // train_generator.batch_size),
                    callbacks = callbacks,
                    validation_data=val_generator,
                    validation_steps=(val_generator.n // val_generator.batch_size),
                    epochs=5,
                    use_multiprocessing=False,
                    workers=8)

filepath = NAME + '.h5'
model.save(filepath)

#### Evaluation

In [ ]:
from sklearn.metrics import confusion_matrix
test_generator.reset()
pred=model.predict_generator(test_generator,
                            steps=test_generator.n//test_generator.batch_size,
                            verbose=1)


In [ ]:
y_true = df_test.gender.astype(int)
y_pred = [1 if x>=0.5 else 0 for x in pred]
cm = confusion_matrix(y_true, y_pred)

In [ ]:
cm

In [ ]:
TN = cm[0][0]
TP = cm[1][1]
FN = cm[1][0]
FP = cm[0][1]

Specificity

In [ ]:
TN/(TN+FP)

Precision

In [ ]:
TP/(TP+FP)

Recall

In [ ]:
TP/FN

Accuracy

In [ ]:
(TP+TN)/(TN+TP+FN+FP)

#### Live Demo

In [ ]:
model = load_model('BN-5-conv-32-node-2-dens-1553895953.h5')

In [ ]:
def prepare(filepath):
    IMG_SIZE = image_reshape_size
    img_array = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
    new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
    return new_array.reshape(-1, IMG_SIZE, IMG_SIZE, 1)

In [ ]:
prediction = model.predict(prepare('picture.jpg'))
print(prediction)